In [2]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
# en_nlp = spacy.load('en')

## Convert Json to Pandas Dataframe

In [8]:
train = pd.read_json("data/BioASQ-train-factoid-7b-full-annotated.json")

In [9]:
valid = pd.read_json("data/BioASQ-train-factoid-7b-full-annotated.json")

In [10]:
train.shape, valid.shape

((1, 2), (1, 2))

In [11]:
train['data'][0].keys()

dict_keys(['paragraphs', 'title'])

In [28]:
len(train['data'][0]['paragraphs'])

5537

In [35]:
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(train.shape[0]):
    topic = train['data'][0]['paragraphs']
    topic = topic[:1000]
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [36]:
df.shape

(1000, 4)

In [37]:
df.to_csv("data/train.csv", index = None)

## Create dictionary of sentence embeddings for faster computation

In [38]:
paras = list(df["context"].drop_duplicates().reset_index(drop= True))

In [39]:
len(paras)

597

In [40]:
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [41]:
len(sentences)

6070

In [42]:
from InferSent import models
MODEL_PATH = "InferSent/encoder/infersent%s.pkl" % 1
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 1}
model = models.InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

# infersent = models.load_state_dict('InferSent/encoder/infersent1.pkl', map_location=lambda storage, loc: storage)
model.set_w2v_path("InferSent/dataset/GloVe/glove_small.840B.300d.txt")

In [43]:
model.build_vocab(sentences, tokenize=True)

Found 3040(/14243) words with w2v vectors
Vocab size : 3040


In [44]:
dict_embeddings = {}
for i in range(len(sentences)):
    print(i)
    dict_embeddings[sentences[i]] = model.encode([sentences[i]], tokenize=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702


5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341


In [45]:
questions = list(df["question"])

In [46]:
len(questions)

1000

In [48]:
for i in range(len(questions)):
    print(i)
    dict_embeddings[questions[i]] = model.encode([questions[i]], tokenize=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [49]:
dict_embeddings['Architecturally, the school has a Catholic character.'][0]

KeyError: 'Architecturally, the school has a Catholic character.'

In [50]:
d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 0}
d2 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 1}

In [51]:
d1

{'Less well appreciated is the fact that within the clinical spectrum of HC are patients who, despite having this disease, have been able to undertake particularly intensive and often extraordinary levels of training for sports competition over many years without dying suddenly.': array([[ 0.03898649,  0.1120668 ,  0.09207065, ...,  0.06455538,
         -0.06825963,  0.00572625]], dtype=float32),
 'Recently, a mutation in the pantothenate kinase (PANK2) gene on band 20pl3 has been described in patients with typical HSD.': array([[ 0.08558047,  0.11086886,  0.11948731, ...,  0.03928574,
         -0.03196335,  0.13339996]], dtype=float32),
 'These results suggest that Mdm2 attenuates TAp73 transactivation function, at least in part, by promoting NEDD8-dependent TAp73 cytoplasmic localization and provide the first evidence of a covalent post-translational modification exclusively targeting the TA isoforms of p73.': array([[ 0.05906054,  0.10006283,  0.09767733, ...,  0.03358415,
         

In [52]:
d2

{'On the other hand, LPS significantly upregulated mRNA expressions and protein levels of Bax and caspase-3 as well as activation of caspase-3, whereas decreased Bcl-2 expression in MC3T3-E1 cells.': array([[ 0.0331207 ,  0.128365  ,  0.0889558 , ...,  0.04612881,
         -0.010423  ,  0.03717127]], dtype=float32),
 'We show that low CCDC6 protein levels are associated with a weak response to DNA damage and a low number of Rad51 positive foci.': array([[ 0.0959838 ,  0.1229398 ,  0.15936035, ...,  0.0946366 ,
         -0.02595175,  0.06923964]], dtype=float32),
 'After 1 year, 100% of AP-CE patients were still alive, compared with 85% of HEM-AP patients and 67.5% of HEM-AP + CE patients (P =.01).': array([[0.01695387, 0.14417909, 0.01831268, ..., 0.14784247, 0.00232462,
         0.07807644]], dtype=float32),
 'PGRN was colocalized with Lamp1, a lysosomal marker, and Lamp1-positive areas in GRN-deficient (KO) mice were significantly expanded compared with wild-type (WT) mice after TBI.

In [53]:
with open('data/dict_embeddings1.pickle', 'wb') as handle:
    pickle.dump(d1, handle)

In [54]:
with open('data/dict_embeddings2.pickle', 'wb') as handle:
    pickle.dump(d2, handle)

In [55]:
del dict_embeddings